In [3]:
# Read json file
import json
import numpy as np
import requests
import os
import pandas as pd
from sklearn import preprocessing
from collections import Counter
from sklearn.decomposition import PCA
from sklearn.cluster import DBSCAN, KMeans
from sklearn.metrics import silhouette_score, calinski_harabasz_score, davies_bouldin_score
import matplotlib.pyplot as plt
import warnings

# Dataframe

In [4]:
playlists_df = pd.read_csv('playlists.csv')
playlists_df = playlists_df.drop(['key'], axis=1)

# Train test split
np.random.seed(42)
msk = np.random.rand(len(playlists_df)) < 0.8
playlists_df, test_playlists_df = playlists_df[msk], playlists_df[~msk]

headers = playlists_df.columns.values.tolist()

print(headers)

['index', 'title', 'danceability', 'energy', 'loudness', 'mode', 'speechiness', 'acousticness', 'instrumentalness', 'liveness', 'valence', 'tempo', 'duration_ms', 'time_signature']


In [5]:
mean = playlists_df[headers[2:]].mean(axis=0)
std = playlists_df[headers[2:]].std(axis=0)

playlists_df[headers[2:]] = (playlists_df[headers[2:]] - mean) / std
test_playlists_df[headers[2:]] = (test_playlists_df[headers[2:]] - mean) / std

In [6]:
playlists = json.load(open('playlists.json', 'r'))
tracks = json.load(open('track_features.json', 'r'))

# Clustering

### K-means

In [7]:
def get_cluster_tracks(playlists_df, cluster_preds, num_clusters, verbose=False):
    global playlists, tracks
    
    playlists_clustered = playlists_df.get(['index', 'title']).copy()
    playlists_clustered['cluster'] = cluster_preds

    cluster_tracks = [{} for _ in range(num_clusters)]
    for cluster in range(num_clusters):
        for i in playlists_clustered[playlists_clustered['cluster'] == cluster]['index']:
            for track in playlists[i][1]:
                if track['track_uri'] not in cluster_tracks[cluster]:
                    cluster_tracks[cluster][track['track_uri']] = track
        cluster_tracks[cluster] = np.array(list(cluster_tracks[cluster].values()))

        if verbose:
            print('Cluster {}: {} tracks'.format(cluster, len(cluster_tracks[cluster])))

    return cluster_tracks

def cluster_custom_score(playlists_df, cluster_preds, num_clusters):
    global tracks
    
    cluster_tracks = get_cluster_tracks(playlists_df, cluster_preds, num_clusters)

    lengths = np.array([len(tracks) for tracks in cluster_tracks])

    return np.sum(lengths) / len(tracks), \
        np.max(lengths) / len(tracks), \
        np.std(lengths) / np.mean(lengths)

In [8]:
NUM_CLUSTERS = 15
kmeans = KMeans(n_clusters=NUM_CLUSTERS, random_state=42, n_init='auto').fit(playlists_df[headers[2:]])

In [9]:
# Group playlists by cluster
playlists_df['cluster'] = kmeans.labels_
playlists_clustered = playlists_df.get(['index', 'title', 'cluster'])

In [10]:
# Count most common titles
def count_words(titles):
    words = []
    for title in titles:
        title = str(title).strip().lower()
        words += title.split(" ")
    return Counter(words)


for i in range(NUM_CLUSTERS):
    print(count_words(
        playlists_clustered[playlists_clustered['cluster'] == i]['title'].values.tolist()).most_common(10))

[('chill', 213), ('summer', 121), ('music', 83), ('good', 83), ('2017', 82), ('2016', 78), ('jams', 76), ('new', 70), ('songs', 64), ('playlist', 59)]
[('christmas', 188), ('disney', 101), ('chill', 70), ('songs', 44), ('music', 43), ('the', 42), ('oldies', 34), ('sleep', 29), ('feels', 29), ('musicals', 23)]
[('rap', 133), ('hop', 79), ('hip', 73), ('old', 59), ('school', 46), ('workout', 23), ('eminem', 22), ('the', 20), ('good', 19), ('throwback', 18)]
[('chill', 40), ('jazz', 16), ('house', 15), ('good', 12), ('2016', 11), ('music', 10), ('the', 10), ('2015', 10), ('electronic', 9), ('study', 8)]
[('classical', 57), ('music', 30), ('study', 29), ('instrumental', 19), ('piano', 16), ('sleep', 16), ('jazz', 13), ('the', 9), ('christmas', 9), ('of', 8)]
[('rock', 166), ('workout', 113), ('edm', 69), ('music', 52), ('playlist', 41), ('my', 39), ('metal', 39), ('new', 31), ('songs', 31), ('gym', 29)]
[('country', 754), ('rock', 142), ('summer', 131), ('songs', 76), ('music', 75), ('play

# Test data

In [11]:
test_playlists_df['cluster'] = kmeans.predict(test_playlists_df[headers[2:]])
test_playlists_clustered = test_playlists_df.get(['index', 'title', 'cluster'])

for i in range(NUM_CLUSTERS):
    print(count_words(
        test_playlists_clustered[test_playlists_clustered['cluster'] == i]['title'].values.tolist()).most_common(10))

[('chill', 56), ('summer', 30), ('2017', 27), ('new', 26), ('vibes', 22), ('good', 19), ('songs', 18), ('2016', 17), ('playlist', 15), ('music', 15)]
[('christmas', 53), ('chill', 24), ('disney', 20), ('songs', 14), ('music', 11), ('sleep', 10), ('sad', 8), ('2017', 7), ('slow', 7), ('oldies', 6)]
[('rap', 26), ('hop', 12), ('old', 12), ('hip', 11), ('school', 9), ('party', 7), ('throwback', 6), ('chill', 5), ('gym', 5), ('workout', 5)]
[('chill', 8), ('house', 4), ('jazz', 4), ('2016', 4), ('playlist', 3), ('2017', 3), ('electronic', 3), ('summer', 3), ('2015', 3), ('music', 3)]
[('study', 14), ('music', 11), ('classical', 10), ('instrumental', 6), ('sleep', 5), ('piano', 5), ('time', 4), ('the', 4), ('movie', 3), ('soundtracks', 3)]
[('workout', 34), ('rock', 32), ('edm', 24), ('music', 13), ('mix', 12), ('songs', 12), ('punk', 8), ('up', 7), ('gym', 7), ('playlist', 7)]
[('country', 182), ('rock', 37), ('summer', 31), ('good', 19), ('new', 17), ('music', 16), ('the', 16), ('playlist

# Get recommendations (for test playlists)

The challenge for the **input** only withholds a constant number of tracks. For simplicity, **we just withhold half the playlist.**

The challenge **output** requires a **list of 500 recommended candidate tracks**, ordered by relevance in decreasing order. We omit the ordering since we do not evaluate the ordering.

In [12]:
cluster_tracks = get_cluster_tracks(playlists_df, playlists_df['cluster'], NUM_CLUSTERS)

cols = ['danceability', 'energy', 'loudness', 'mode', 'speechiness',
        'acousticness', 'instrumentalness', 'liveness', 'valence', 'tempo',
        'duration_ms', 'time_signature']

In [13]:
def get_playlist_features(playlist_tracks, mean=mean, std=std):
    features = [np.mean([track[col] for track in playlist_tracks]) for col in cols]
    return np.array((features - mean) / std)

def get_track_info(tracks):
    return np.array([[s['artist_name'], s['track_name']] for s in tracks])

def get_track_features(tracks, mean=mean, std=std):
    features = np.array([[s[col] for col in cols] for s in tracks])
    return np.array((features - np.array(mean)) / np.array(std))

pretty_print = lambda x: "{} - {}".format(x[0], x[1])

In [146]:
def k_nn(k, needle_features, haystack_features):
    """
    Given an instance of features, find the k nearest neighbors in the haystack. 
    Return indexes within haystack.
    """
    distances = np.linalg.norm(needle_features - haystack_features, axis=1)
    return np.argsort(distances)[:k]

In [147]:
def r_precision(pred_tracks: set, target_tracks: set):
    return len(pred_tracks.intersection(target_tracks)) / len(target_tracks)

Accuracy of clustering based on half the playlist

In [126]:
PART_PERCENT = 0.5 # Percentage of playlist to use for clustering

count = 0
for test_i in range(len(test_playlists_df)):
    i = test_playlists_df.iloc[test_i]['index']

    playlist_tracks = playlists[i][1]
    np.random.shuffle(playlist_tracks) # Shuffle tracks - improved accuracy

    playlist_part_features = get_playlist_features(
        playlist_tracks[:int(len(playlist_tracks) * PART_PERCENT)])

    playlist_features = np.array(test_playlists_df.iloc[test_i][headers[2:]].values, dtype='float32')

    with warnings.catch_warnings():
        warnings.simplefilter("ignore")
        real_cluster = kmeans.predict([playlist_features])[0]
        part_cluster = kmeans.predict([playlist_part_features])[0]

    count += 1 if real_cluster == part_cluster else 0

print('Accuracy by clustering using only {}% of the playlist: {:.2f}%'.format(
    PART_PERCENT*100, 100*count / len(test_playlists_df)))

Accuracy by clustering using only 50.0% of the playlist: 52.67%


Sample from nearest random playlists

In [234]:
test_i = np.random.randint(len(test_playlists_df))
i = test_playlists_df.iloc[test_i]['index']

playlist_name = playlists[i][0]
playlist_tracks = playlists[i][1]
np.random.shuffle(playlist_tracks)

playlist_part_tracks = playlist_tracks[:int(len(playlists[i][1]) * PART_PERCENT)]

playlist_part_features = get_playlist_features(playlist_part_tracks)

playlist_features = np.array(test_playlists_df.iloc[test_i][headers[2:]].values, dtype='float32')

with warnings.catch_warnings():
    warnings.simplefilter("ignore")
    real_cluster = kmeans.predict([playlist_features])[0]
    part_cluster = kmeans.predict([playlist_part_features])[0]

tracks_in_cluster = cluster_tracks[part_cluster]
tracks_info = get_track_info(tracks_in_cluster)
track_features = get_track_features(tracks_in_cluster)

print("Playlist name: {}".format(playlist_name))
print("Number of tracks selected: {}".format(len(playlist_part_tracks)))
print("Number of tracks omitted: {}".format(len(playlist_tracks) - len(playlist_part_tracks)))
print("Real cluster: {}".format(real_cluster))
print("Part cluster: {}".format(part_cluster))
print("Tracks selected (first 10): ")

print('\t' + '\n\t'.join(map(pretty_print, get_track_info(playlist_part_tracks[:10]))))

Playlist name: christmas 2017
Number of tracks selected: 86
Number of tracks omitted: 86
Real cluster: 1
Part cluster: 1
Tracks selected (first 10): 
	Elvis Presley - O Little Town of Bethlehem
	Frank Sinatra - Jingle Bells - 1999 - Remaster
	William Bell - Everyday Will Be Like A Holiday - Single Version
	Weezer - We Wish You A Merry Christmas
	Peggy Lee - It's Christmas Time Again
	Frank Sinatra - Mistletoe And Holly - 1999 - Remaster
	Weezer - Silent Night
	Jimmy Smith - Silent Night
	Ella Fitzgerald - Winter Wonderland
	Ella Fitzgerald - What Are You Doing New Year's Eve?


K-nn to find nearest playlists (then randomly sample tracks)

1. View the nearest playlist

In [235]:
# Playlists (from training data) in the same cluster (predicted from part of the playlist)
haystack_playlists = playlists_df[playlists_df['cluster'] == part_cluster] 
haystack_features = haystack_playlists[headers[2:]].values

nearest_playlists = k_nn(500, playlist_part_features, haystack_features)
i = nearest_playlists[0]

pred_playlist = haystack_playlists.iloc[i]
i = pred_playlist['index']

pred_playlist_name = playlists[i][0]
pred_playlist_tracks = playlists[i][1]
np.random.shuffle(pred_playlist_tracks)
pred_num_tracks = len(playlists[i][1])

print("Playlist name: {}".format(pred_playlist_name))
print("Number of tracks: {}".format(pred_num_tracks))
print("Tracks (random 10): ")
print('\t' + '\n\t'.join(map(pretty_print, get_track_info(pred_playlist_tracks[:10]))))

Playlist name: Christmas
Number of tracks: 159
Tracks (random 10): 
	Paul McCartney - Wonderful Christmastime - Remastered 2011 / Edited Version
	Josh Groban - Silent Night
	Vince Guaraldi Trio - Skating
	Vince Guaraldi Trio - Christmas Time Is Here - Instrumental
	Michael Bublé - The Christmas Song
	Pentatonix - Joy to the World
	Perry Como - (There's No Place Like) Home for the Holidays - 1954 Version
	Frank Sinatra - Winter Wonderland
	Pentatonix - Silent Night
	Pentatonix - Little Drummer Boy


2. Get 500 songs

In [236]:
init_tracks = set([track['track_uri'] for track in playlist_part_tracks])
pred_tracks = init_tracks.copy()

for i in nearest_playlists:
    pred_playlist = haystack_playlists.iloc[i]
    i = pred_playlist['index']
    for uri in [track['track_uri'] for track in playlists[i][1]]:
        pred_tracks.add(uri)

        if len(pred_tracks) >= 500 + len(init_tracks):
            break

    if len(pred_tracks) >= 500 + len(init_tracks):
        break

pred_tracks = pred_tracks - init_tracks

target_tracks = set([track['track_uri'] for track in playlist_tracks]) - init_tracks
print("R-precision: {:.2f}%".format(100 * r_precision(pred_tracks, target_tracks)))

R-precision: 27.91%


K-NN to find tracks nearest to playlist aggregate features

1. View nearest 10

In [237]:
playlist_part_features_pred = [tracks_info[i] for i in k_nn(10, playlist_part_features, track_features)]
print("Playlist part feature predictions: \n{}\n".format("\n".join(map(pretty_print, playlist_part_features_pred))))

# playlist_features_pred = [tracks_info[i] for i in k_nn(10, playlist_features, track_features)]
# print("Playlist feature predictions: \n{}".format("\n".join(map(pretty_print, playlist_features_pred))))

Playlist part feature predictions: 
Johnny Mathis - The Very First Christmas Day - Single Version
The Milk Carton Kids - High Hopes
Johnny Bond - Stars Of The Midnight Range
Astrud Gilberto - Misty Roses
The Weepies - World Spins Madly On
Ricky Valance - Tell Laura I Love Her
Simon & Garfunkel - Flowers Never Bend with the Rainfall
Frank Sinatra - Wave
Dean Martin - Red Roses for a Blue Lady
Frank Sinatra - L.A. Is My Lady



2. Get 500

In [238]:
pred_tracks = init_tracks.copy()

for i in k_nn(500 + len(init_tracks), playlist_part_features, track_features):
    pred_tracks.add(tracks_in_cluster[i]['track_uri'])

    if len(pred_tracks) >= 500 + len(init_tracks):
        break

pred_tracks = pred_tracks - init_tracks

print("R-precision: {:.2f}%".format(100 * r_precision(pred_tracks, target_tracks)))

R-precision: 1.16%


K-NN to find tracks nearest to random tracks in playlist

1. View song to song predictions

In [239]:
random_tracks = np.random.choice(playlist_part_tracks, 10, replace=False)

random_tracks_info = get_track_info(random_tracks)
random_track_features = get_track_features(random_tracks)

random_tracks_pred = []
for features in random_track_features:
    random_tracks_pred.append(tracks_info[k_nn(2, features, track_features)[1]])

for random_track, random_track_pred in zip(random_tracks_info, random_tracks_pred):
    print("{} ->\n\t{}".format(pretty_print(random_track), pretty_print(random_track_pred)))

James Brown - Let's Make Christmas Mean Something This Year - Pts. 1 & 2 ->
	Patrick Sweany - Every Night Every Day
Elvis Presley - O Little Town of Bethlehem ->
	Amanda Seyfried - In My Life / A Heart Full Of Love
The Crystals - Parade Of The Wooden Soldiers ->
	alt-J - Every Other Freckle
The Ronettes - I Saw Mommy Kissing Santa Claus ->
	Charles Aznavour - De t’avoir aimée
Mahalia Jackson - Silent Night, Holy Night ->
	Barry Manilow - We Wish You A Merry Christmas/It's Just Another New Year's Eve
Carpenters - Santa Claus Is Coming To Town - Single Version ->
	The Avett Brothers - Pretty Girl From Locust
The Beach Boys - Merry Christmas, Baby ->
	Lesley Gore - It's My Party
The Crystals - Rudolph the Red-Nosed Reindeer ->
	Barry Manilow - Happy Holiday/White Christmas
Ella Fitzgerald - What Are You Doing New Year's Eve? ->
	Sara Bareilles - Bluebird
Ella Fitzgerald - Have Yourself A Merry Little Christmas ->
	Hank Thompson - The Wild Side Of Life


2. Get 500

In [240]:
pred_tracks = init_tracks.copy()

playlist_track_features = [get_track_features([track])[0] for track in playlist_part_tracks]
playlist_track_nn = [k_nn(len(track_features), features, track_features) for features in playlist_track_features]

for i in range(500):
    for track_nn in playlist_track_nn:
        pred_tracks.add(tracks_in_cluster[track_nn[i]]['track_uri'])

        if len(pred_tracks) >= 500 + len(init_tracks):
            break

    if len(pred_tracks) >= 500 + len(init_tracks):
        break

pred_tracks = pred_tracks - init_tracks

print("R-precision: {:.2f}%".format(100 * r_precision(pred_tracks, target_tracks)))

R-precision: 6.98%
